# Option Strategy Backtesting

This notebook covers:
1. Fetching historical market data
2. Simulating common option strategies
3. Performance metrics and risk analysis
4. Strategy comparison
5. Trade execution and rolling positions
6. Practical insights for live trading

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our custom pricing functions
import sys
sys.path.append('../src')
from options_desk.pricing.black_scholes import (
    black_scholes_price,
    black_scholes_delta
)
from options_desk.core.option import OptionType

## 1. Fetch Historical Data

We'll use historical stock price data to simulate option strategies.

In [ ]:
# Fetch historical data
TICKER = 'SPY'
START_DATE = '2020-01-01'
END_DATE = '2024-01-01'

ticker = yf.Ticker(TICKER)
hist_data = ticker.history(start=START_DATE, end=END_DATE)

print(f"Historical Data for {TICKER}")
print(f"Period: {START_DATE} to {END_DATE}")
print(f"Trading Days: {len(hist_data)}")
print(f"\nFirst few rows:")
hist_data.head()

In [ ]:
# Plot historical prices
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Price chart
axes[0].plot(hist_data.index, hist_data['Close'], linewidth=1.5)
axes[0].set_ylabel('Price ($)', fontsize=12)
axes[0].set_title(f'{TICKER} Historical Price', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Daily returns
hist_data['Returns'] = hist_data['Close'].pct_change()
axes[1].plot(hist_data.index, hist_data['Returns'] * 100, linewidth=0.8, alpha=0.7)
axes[1].axhline(0, color='black', linestyle='-', linewidth=1)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Daily Return (%)', fontsize=12)
axes[1].set_title('Daily Returns', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate historical volatility
hist_data['RollingVol'] = hist_data['Returns'].rolling(window=21).std() * np.sqrt(252) * 100

print(f"\nHistorical Volatility Statistics:")
print(f"  Mean: {hist_data['RollingVol'].mean():.2f}%")
print(f"  Min: {hist_data['RollingVol'].min():.2f}%")
print(f"  Max: {hist_data['RollingVol'].max():.2f}%")

## 2. Strategy Framework

Define base class for option strategies.

In [ ]:
class OptionStrategy:
    """
    Base class for option strategies.
    """
    def __init__(self, name, initial_capital=10000):
        self.name = name
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.positions = []
        self.equity_curve = []
        self.trades = []
    
    def calculate_payoff(self, spot, strike, option_type, position):
        """
        Calculate option payoff at expiration.
        position: 'LONG' or 'SHORT'
        """
        if option_type == 'CALL':
            intrinsic = max(spot - strike, 0)
        else:  # PUT
            intrinsic = max(strike - spot, 0)
        
        return intrinsic if position == 'LONG' else -intrinsic
    
    def execute_strategy(self, hist_data):
        """
        Override in subclass.
        """
        raise NotImplementedError
    
    def calculate_metrics(self):
        """
        Calculate performance metrics.
        """
        equity_series = pd.Series(self.equity_curve)
        returns = equity_series.pct_change().dropna()
        
        total_return = (equity_series.iloc[-1] / equity_series.iloc[0] - 1) * 100
        annual_return = ((equity_series.iloc[-1] / equity_series.iloc[0]) ** 
                        (252 / len(equity_series)) - 1) * 100
        
        volatility = returns.std() * np.sqrt(252) * 100
        sharpe = (annual_return - 4.5) / volatility if volatility > 0 else 0  # 4.5% risk-free
        
        # Max drawdown
        cummax = equity_series.cummax()
        drawdown = (equity_series - cummax) / cummax * 100
        max_drawdown = drawdown.min()
        
        # Win rate
        trades_df = pd.DataFrame(self.trades)
        win_rate = (trades_df['pnl'] > 0).mean() * 100 if len(trades_df) > 0 else 0
        
        return {
            'total_return': total_return,
            'annual_return': annual_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'num_trades': len(trades_df)
        }

## 3. Strategy 1: Covered Call

**Setup:** Long stock + Short OTM call  
**Goal:** Generate income from selling calls while holding stock  
**Risk:** Upside capped at strike price

In [ ]:
class CoveredCall(OptionStrategy):
    def __init__(self, otm_pct=0.05, days_to_expiry=30, initial_capital=10000):
        super().__init__('Covered Call', initial_capital)
        self.otm_pct = otm_pct  # Strike is spot * (1 + otm_pct)
        self.days_to_expiry = days_to_expiry
    
    def execute_strategy(self, hist_data, volatility=0.20, risk_free_rate=0.045):
        """
        Execute covered call strategy.
        Roll position every month.
        """
        data = hist_data.copy()
        data['Date'] = data.index
        
        # Buy stock initially
        entry_price = data['Close'].iloc[0]
        num_shares = int(self.capital / entry_price)
        stock_cost = num_shares * entry_price
        self.capital -= stock_cost
        
        position_open = False
        entry_date = None
        strike = None
        option_premium = 0
        
        for i, (date, row) in enumerate(data.iterrows()):
            spot = row['Close']
            
            # Calculate equity (stock value + cash)
            stock_value = num_shares * spot
            equity = stock_value + self.capital
            self.equity_curve.append(equity)
            
            # Open new position every month
            if not position_open:
                # Sell OTM call
                strike = spot * (1 + self.otm_pct)
                time_to_expiry = self.days_to_expiry / 365.25
                
                # Price the call option
                option_premium = black_scholes_price(
                    spot, strike, time_to_expiry, risk_free_rate, 
                    volatility, OptionType.CALL
                ) * num_shares * 100  # Per share * shares * multiplier
                
                # Receive premium
                self.capital += option_premium
                
                entry_date = date
                entry_spot = spot
                position_open = True
            
            # Check if position should be closed (30 days elapsed)
            if position_open and i > 0:
                days_elapsed = (date - entry_date).days
                
                if days_elapsed >= self.days_to_expiry:
                    # Close position
                    if spot > strike:
                        # Shares called away
                        stock_sale = num_shares * strike
                        self.capital += stock_sale
                        
                        # Buy back shares at current price
                        stock_cost = num_shares * spot
                        self.capital -= stock_cost
                        
                        pnl = option_premium + (strike - entry_spot) * num_shares
                    else:
                        # Option expires worthless, keep shares and premium
                        pnl = option_premium + (spot - entry_spot) * num_shares
                    
                    self.trades.append({
                        'entry_date': entry_date,
                        'exit_date': date,
                        'entry_price': entry_spot,
                        'exit_price': spot,
                        'strike': strike,
                        'premium': option_premium,
                        'pnl': pnl
                    })
                    
                    position_open = False
        
        return self

## 4. Strategy 2: Cash-Secured Put

**Setup:** Short OTM put + hold cash to buy stock if assigned  
**Goal:** Generate income, potentially buy stock at lower price  
**Risk:** Forced to buy stock if it falls below strike

In [ ]:
class CashSecuredPut(OptionStrategy):
    def __init__(self, otm_pct=0.05, days_to_expiry=30, initial_capital=10000):
        super().__init__('Cash-Secured Put', initial_capital)
        self.otm_pct = otm_pct
        self.days_to_expiry = days_to_expiry
    
    def execute_strategy(self, hist_data, volatility=0.20, risk_free_rate=0.045):
        data = hist_data.copy()
        
        position_open = False
        entry_date = None
        strike = None
        num_contracts = 1
        
        for i, (date, row) in enumerate(data.iterrows()):
            spot = row['Close']
            self.equity_curve.append(self.capital)
            
            if not position_open:
                # Sell OTM put
                strike = spot * (1 - self.otm_pct)
                time_to_expiry = self.days_to_expiry / 365.25
                
                option_premium = black_scholes_price(
                    spot, strike, time_to_expiry, risk_free_rate,
                    volatility, OptionType.PUT
                ) * num_contracts * 100
                
                self.capital += option_premium
                entry_date = date
                entry_spot = spot
                entry_premium = option_premium
                position_open = True
            
            if position_open and i > 0:
                days_elapsed = (date - entry_date).days
                
                if days_elapsed >= self.days_to_expiry:
                    # Close position
                    if spot < strike:
                        # Assigned - buy stock at strike
                        assignment_loss = (strike - spot) * num_contracts * 100
                        pnl = entry_premium - assignment_loss
                    else:
                        # Expire worthless - keep premium
                        pnl = entry_premium
                    
                    self.trades.append({
                        'entry_date': entry_date,
                        'exit_date': date,
                        'entry_price': entry_spot,
                        'exit_price': spot,
                        'strike': strike,
                        'premium': entry_premium,
                        'pnl': pnl
                    })
                    
                    position_open = False
        
        return self

## 5. Strategy 3: Iron Condor

**Setup:** Sell OTM call spread + Sell OTM put spread  
**Goal:** Profit from low volatility, range-bound market  
**Risk:** Large moves in either direction

In [ ]:
class IronCondor(OptionStrategy):
    def __init__(self, width=0.05, days_to_expiry=30, initial_capital=10000):
        super().__init__('Iron Condor', initial_capital)
        self.width = width
        self.days_to_expiry = days_to_expiry
    
    def execute_strategy(self, hist_data, volatility=0.20, risk_free_rate=0.045):
        data = hist_data.copy()
        
        position_open = False
        
        for i, (date, row) in enumerate(data.iterrows()):
            spot = row['Close']
            self.equity_curve.append(self.capital)
            
            if not position_open:
                # Define strikes
                put_short_strike = spot * (1 - self.width)
                put_long_strike = spot * (1 - 2 * self.width)
                call_short_strike = spot * (1 + self.width)
                call_long_strike = spot * (1 + 2 * self.width)
                
                time_to_expiry = self.days_to_expiry / 365.25
                
                # Calculate premiums (100 multiplier)
                put_short_premium = black_scholes_price(
                    spot, put_short_strike, time_to_expiry, 
                    risk_free_rate, volatility, OptionType.PUT) * 100
                put_long_premium = black_scholes_price(
                    spot, put_long_strike, time_to_expiry,
                    risk_free_rate, volatility, OptionType.PUT) * 100
                call_short_premium = black_scholes_price(
                    spot, call_short_strike, time_to_expiry,
                    risk_free_rate, volatility, OptionType.CALL) * 100
                call_long_premium = black_scholes_price(
                    spot, call_long_strike, time_to_expiry,
                    risk_free_rate, volatility, OptionType.CALL) * 100
                
                # Net credit received
                net_premium = (put_short_premium - put_long_premium + 
                              call_short_premium - call_long_premium)
                
                self.capital += net_premium
                entry_date = date
                entry_spot = spot
                position_open = True
                
                # Store strikes
                strikes = {
                    'put_long': put_long_strike,
                    'put_short': put_short_strike,
                    'call_short': call_short_strike,
                    'call_long': call_long_strike
                }
            
            if position_open and i > 0:
                days_elapsed = (date - entry_date).days
                
                if days_elapsed >= self.days_to_expiry:
                    # Calculate P&L at expiration
                    put_spread_pnl = 0
                    call_spread_pnl = 0
                    
                    # Put spread
                    if spot < strikes['put_short']:
                        if spot < strikes['put_long']:
                            # Max loss
                            put_spread_pnl = -(strikes['put_short'] - strikes['put_long']) * 100
                        else:
                            # Partial loss
                            put_spread_pnl = -(strikes['put_short'] - spot) * 100
                    
                    # Call spread
                    if spot > strikes['call_short']:
                        if spot > strikes['call_long']:
                            # Max loss
                            call_spread_pnl = -(strikes['call_long'] - strikes['call_short']) * 100
                        else:
                            # Partial loss
                            call_spread_pnl = -(spot - strikes['call_short']) * 100
                    
                    total_pnl = net_premium + put_spread_pnl + call_spread_pnl
                    
                    self.trades.append({
                        'entry_date': entry_date,
                        'exit_date': date,
                        'entry_price': entry_spot,
                        'exit_price': spot,
                        'premium': net_premium,
                        'pnl': total_pnl
                    })
                    
                    self.capital += put_spread_pnl + call_spread_pnl
                    position_open = False
        
        return self

## 6. Run Backtests

In [ ]:
# Initialize strategies
strategies = [
    CoveredCall(otm_pct=0.05, days_to_expiry=30),
    CashSecuredPut(otm_pct=0.05, days_to_expiry=30),
    IronCondor(width=0.05, days_to_expiry=30)
]

# Run backtests
print("Running backtests...\n")
results = {}

for strategy in strategies:
    print(f"Backtesting {strategy.name}...")
    strategy.execute_strategy(hist_data, volatility=0.20)
    metrics = strategy.calculate_metrics()
    results[strategy.name] = {
        'strategy': strategy,
        'metrics': metrics
    }
    print(f"  Completed: {metrics['num_trades']} trades")

print("\nBacktesting complete!")

## 7. Performance Comparison

In [ ]:
# Display results table
metrics_df = pd.DataFrame({
    name: data['metrics'] 
    for name, data in results.items()
}).T

print("\n" + "="*80)
print("STRATEGY PERFORMANCE COMPARISON")
print("="*80)
print(metrics_df.to_string())

# Add benchmark (buy and hold)
buy_hold_return = (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100
print(f"\nBenchmark (Buy & Hold {TICKER}): {buy_hold_return:.2f}%")

In [ ]:
# Plot equity curves
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Equity curves
ax1 = axes[0, 0]
for name, data in results.items():
    equity = data['strategy'].equity_curve
    dates = hist_data.index[:len(equity)]
    ax1.plot(dates, equity, label=name, linewidth=2)

# Add buy & hold
initial_capital = 10000
buy_hold_equity = (hist_data['Close'] / hist_data['Close'].iloc[0]) * initial_capital
ax1.plot(hist_data.index, buy_hold_equity, label='Buy & Hold', 
         linewidth=2, linestyle='--', alpha=0.7)

ax1.set_xlabel('Date', fontsize=11)
ax1.set_ylabel('Portfolio Value ($)', fontsize=11)
ax1.set_title('Equity Curves', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Returns comparison
ax2 = axes[0, 1]
returns_data = [data['metrics']['annual_return'] for data in results.values()]
returns_data.append(buy_hold_return)
labels = list(results.keys()) + ['Buy & Hold']
colors = ['green', 'blue', 'purple', 'gray']
ax2.bar(labels, returns_data, color=colors, alpha=0.7, edgecolor='black')
ax2.axhline(0, color='black', linestyle='-', linewidth=1)
ax2.set_ylabel('Annual Return (%)', fontsize=11)
ax2.set_title('Annual Returns', fontsize=13, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

# Sharpe ratios
ax3 = axes[1, 0]
sharpe_data = [data['metrics']['sharpe_ratio'] for data in results.values()]
ax3.bar(list(results.keys()), sharpe_data, color=colors[:-1], alpha=0.7, edgecolor='black')
ax3.axhline(0, color='black', linestyle='-', linewidth=1)
ax3.set_ylabel('Sharpe Ratio', fontsize=11)
ax3.set_title('Risk-Adjusted Returns (Sharpe Ratio)', fontsize=13, fontweight='bold')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# Max drawdown
ax4 = axes[1, 1]
dd_data = [data['metrics']['max_drawdown'] for data in results.values()]
ax4.bar(list(results.keys()), dd_data, color=colors[:-1], alpha=0.7, edgecolor='black')
ax4.set_ylabel('Max Drawdown (%)', fontsize=11)
ax4.set_title('Maximum Drawdown', fontsize=13, fontweight='bold')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 8. Trade Analysis

In [ ]:
# Analyze individual trades for each strategy
for name, data in results.items():
    strategy = data['strategy']
    trades_df = pd.DataFrame(strategy.trades)
    
    if len(trades_df) > 0:
        print(f"\n{'='*70}")
        print(f"{name} - Trade Statistics")
        print(f"{'='*70}")
        print(f"Total Trades: {len(trades_df)}")
        print(f"Winning Trades: {(trades_df['pnl'] > 0).sum()}")
        print(f"Losing Trades: {(trades_df['pnl'] < 0).sum()}")
        print(f"Win Rate: {(trades_df['pnl'] > 0).mean() * 100:.1f}%")
        print(f"\nP&L Statistics:")
        print(f"  Average P&L: ${trades_df['pnl'].mean():.2f}")
        print(f"  Best Trade: ${trades_df['pnl'].max():.2f}")
        print(f"  Worst Trade: ${trades_df['pnl'].min():.2f}")
        print(f"  Total P&L: ${trades_df['pnl'].sum():.2f}")
        
        # Plot P&L distribution
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.hist(trades_df['pnl'], bins=30, alpha=0.7, edgecolor='black')
        ax.axvline(trades_df['pnl'].mean(), color='red', linestyle='--', 
                   linewidth=2, label=f"Mean: ${trades_df['pnl'].mean():.2f}")
        ax.axvline(0, color='black', linestyle='-', linewidth=1)
        ax.set_xlabel('P&L per Trade ($)', fontsize=11)
        ax.set_ylabel('Frequency', fontsize=11)
        ax.set_title(f'{name} - Trade P&L Distribution', fontsize=13, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## 9. Summary and Insights

### Key Takeaways:

1. **Covered Call:**
   - Generates steady income from selling calls
   - Limits upside potential (capped at strike)
   - Suitable for neutral to slightly bullish outlook
   - Lower volatility than buy & hold

2. **Cash-Secured Put:**
   - Collects premium for accepting downside risk
   - Can acquire stock at discount if assigned
   - Works best in stable or rising markets
   - Loses money in sharp downturns

3. **Iron Condor:**
   - Profits from low volatility and range-bound markets
   - Defined risk (max loss = width of spread)
   - High win rate but smaller gains per trade
   - Vulnerable to large unexpected moves

### Practical Considerations:

1. **Transaction Costs:** Real trading has commissions and bid-ask spreads
2. **Slippage:** Market impact when entering/exiting positions
3. **Implied Volatility:** Market IV varies over time (VIX)
4. **Dividends:** Affect option pricing and early exercise
5. **Early Assignment:** Possible for ITM options before expiry
6. **Position Sizing:** Don't risk too much on single trade

### Next Steps:

1. Test strategies with different parameters (strike, expiry, width)
2. Incorporate real option data instead of Black-Scholes pricing
3. Add transaction costs and slippage
4. Test on different market regimes (bull, bear, sideways)
5. Implement dynamic volatility estimation
6. Consider portfolio of multiple strategies